In [1]:
import pandas as pd
import dask.dataframe as dd

In [2]:
import warnings 
warnings.filterwarnings('ignore')

In [3]:
df1 = pd.read_csv('data_승인횟수.csv')
df2 = pd.read_csv('data_요구불.csv')

In [4]:
df3 = pd.read_csv('df.csv')

In [4]:
df1

,거래년월,고객ID,승인횟수
0,202112,8148085642,2.5
1,202112,2800789394,2.5
2,202112,7428017497,2.5
3,202112,3926398218,2.5
4,202112,6773300779,2.5
...,...,...,...
43831120,202102,4203651799,2.5
43831121,202101,9086049340,2.5
43831122,202101,6681903920,2.5
43831123,202101,6349438679,2.5


In [5]:
df2

,고객ID,수신_요구불예금,기준년월,연령,성별
0,2800584251,0.000000e+00,202112,20,남
1,2800585236,0.000000e+00,202112,30,남
2,2800588947,1.000000e+05,202112,30,남
3,2800589092,0.000000e+00,202112,40,여
4,2800597997,0.000000e+00,202112,30,남
...,...,...,...,...,...
1781901,9999988325,1.952174e+06,202110,60,남
1781902,9999989448,4.000000e+05,202110,60,남
1781903,9999991579,0.000000e+00,202110,60,여
1781904,9999997589,0.000000e+00,202110,60,남


In [5]:
df1 = df1.rename(columns={'거래년월': '년월'})
df2 = df2.rename(columns={'기준년월': '년월'})

merged_df = pd.merge(df1, df2, on=['고객ID', '년월'], how='left')

In [9]:
merged_df

,년월,고객ID,승인횟수,수신_요구불예금,연령,성별
0,202112,8148085642,2.5,NaN,NaN,NaN
1,202112,2800789394,2.5,0.000000e+00,50.0,여
2,202112,7428017497,2.5,NaN,NaN,NaN
3,202112,3926398218,2.5,NaN,NaN,NaN
4,202112,6773300779,2.5,2.683333e+06,50.0,여
...,...,...,...,...,...,...
43831120,202102,4203651799,2.5,NaN,NaN,NaN
43831121,202101,9086049340,2.5,NaN,NaN,NaN
43831122,202101,6681903920,2.5,NaN,NaN,NaN
43831123,202101,6349438679,2.5,NaN,NaN,NaN


# 주거래

In [6]:
filtered_df = merged_df[merged_df['수신_요구불예금'].notnull()]

In [7]:
filtered_df = filtered_df[(filtered_df['승인횟수'] >= 15 )]

In [9]:
filtered_df['년월'] = filtered_df['년월'].astype(int)
filtered_df = filtered_df.loc[filtered_df.groupby('고객ID')['년월'].idxmax()]

In [10]:
filtered_df

,년월,고객ID,승인횟수,수신_요구불예금,연령,성별
11939,202112,2814287598,22.5,0.000000e+00,10.0,남
17867,202112,2815976612,17.5,1.000000e+05,40.0,남
19235,202112,2816251939,22.5,3.000000e+05,10.0,남
19292,202112,2816253668,22.5,8.000000e+05,20.0,남
26246390,202112,2817396105,17.5,6.500000e+05,40.0,남
...,...,...,...,...,...,...
25759676,202110,9997197549,22.5,7.086957e+05,40.0,남
992908,202110,9997541878,22.5,1.747826e+06,50.0,여
33739253,202110,9997848850,22.5,3.521739e+05,50.0,여
13665098,202110,9998496340,17.5,5.565217e+05,50.0,남


- customer data가 없는 사람들 어떻게?
- 진짜 주거래임을 어떻게 증명?

# 주거래2

In [49]:
df = df3[['고객ID', '월평균 거래건수','최다가맹점업종명','업종다양성지수','거래 밀도 지수','거래활동기간','거래 연속성 비율','총거래건수']]
#거래밀도지수 : 한달 평균 몇건

In [50]:
df['업종다양성지수'] = df['업종다양성지수'].astype(int)
df['월평균 거래건수'] = df['월평균 거래건수'].astype(int)
df['거래활동기간'] = df['거래활동기간'].astype(int)
df['거래 연속성 비율'] = df['거래 연속성 비율'].astype(int)
df['총거래건수'] = df['총거래건수'].astype(int)

In [51]:
print(df.dtypes)

고객ID           int64
월평균 거래건수       int32
최다가맹점업종명      object
업종다양성지수        int32
거래 밀도 지수     float64
거래활동기간         int32
거래 연속성 비율      int32
총거래건수          int32
dtype: object


In [52]:
f_d = df.query(
    "`업종다양성지수` >= 14 and `월평균 거래건수` >= 20 and `거래활동기간` >= 25 and `거래 연속성 비율` >= 97 and `총거래건수` >= 540"
)

In [56]:
f_d

,고객ID,월평균 거래건수,최다가맹점업종명,업종다양성지수,거래 밀도 지수,거래활동기간,거래 연속성 비율,총거래건수
24637,2802625115,52,편 의 점,46,52.60,25,100,1315
24673,2803501424,69,인터넷P/G,40,69.50,25,100,1737
24682,2803619663,80,일반한식,57,80.10,25,100,2002
24705,2804363794,46,서양음식,32,46.40,25,100,1160
24731,2804602026,37,서양음식,40,37.30,25,100,932
...,...,...,...,...,...,...,...,...
915180,9999486316,32,약 국,30,32.22,36,100,1160
915187,9999556499,49,일반한식,34,49.44,36,100,1780
915190,9999569060,87,일반한식,64,87.43,36,100,3147
915194,9999679820,29,일반한식,42,29.51,36,100,1062


In [57]:
f_d.to_csv('주거래.csv', encoding='utf-8', index=False)

# 부거래
주거래랑 다른 방식으로 진행

In [22]:
df3 #카드 고객 기준

,고객ID,최초거래년월,최종거래년월,거래활동기간,거래 연속성 비율,거래 밀도 지수,비활성화 기간 비율,총 거래 공백 개월수,월평균 거래금액,월평균 거래건수,최다가맹점업종명,업종다양성지수,신규유입고객,최다가맹점_광역시도,거래지역다양성지수,총거래금액,총거래건수
0,2002967708,202312,202312,1,100.00,5.00,0.00,0,210000.00,5.00,대형할인점,2,1,경상북도,2,210000,5.0
1,2004066711,202312,202312,1,100.00,2.50,0.00,0,40000.00,2.50,대형할인점,1,1,경기도,1,40000,2.5
2,2004166649,202312,202312,1,100.00,15.00,0.00,0,220000.00,15.00,기타잡화,6,1,경상북도,3,220000,15.0
3,2004326617,202312,202312,1,100.00,2.50,0.00,0,10000.00,2.50,편 의 점,1,1,경기도,1,10000,2.5
4,2004432929,202312,202312,1,100.00,5.00,0.00,0,460000.00,5.00,대형할인점,2,1,경기도,2,460000,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915212,9999918728,202105,202310,30,43.33,4.23,0.57,17,443076.92,4.23,대형할인점,6,1,대구광역시,1,5760000,55.0
915213,9999933174,202109,202212,16,43.75,9.29,0.56,9,611428.57,9.29,약 국,14,1,경기도,4,4280000,65.0
915214,9999972286,202101,202101,1,100.00,2.50,0.00,0,10000.00,2.50,편 의 점,1,0,대구광역시,1,10000,2.5
915215,9999981183,202101,202211,23,100.00,21.20,0.00,0,371304.35,21.20,일반한식,31,0,경상북도,6,8540000,487.5


In [58]:
data = df3[['고객ID', '월평균 거래건수','최다가맹점업종명','업종다양성지수','거래 밀도 지수','거래활동기간','거래 연속성 비율','총거래건수']]

In [59]:
data

,고객ID,월평균 거래건수,최다가맹점업종명,업종다양성지수,거래 밀도 지수,거래활동기간,거래 연속성 비율,총거래건수
0,2002967708,5.00,대형할인점,2,5.00,1,100.00,5.0
1,2004066711,2.50,대형할인점,1,2.50,1,100.00,2.5
2,2004166649,15.00,기타잡화,6,15.00,1,100.00,15.0
3,2004326617,2.50,편 의 점,1,2.50,1,100.00,2.5
4,2004432929,5.00,대형할인점,2,5.00,1,100.00,5.0
...,...,...,...,...,...,...,...,...
915212,9999918728,4.23,대형할인점,6,4.23,30,43.33,55.0
915213,9999933174,9.29,약 국,14,9.29,16,43.75,65.0
915214,9999972286,2.50,편 의 점,1,2.50,1,100.00,2.5
915215,9999981183,21.20,일반한식,31,21.20,23,100.00,487.5


In [60]:
data['업종다양성지수'] = data['업종다양성지수'].astype(int)
data['월평균 거래건수'] = data['월평균 거래건수'].astype(int)
data['거래활동기간'] = data['거래활동기간'].astype(int)
data['거래 연속성 비율'] = data['거래 연속성 비율'].astype(int)
data['총거래건수'] = data['총거래건수'].astype(int)

In [62]:
filtered_df = df.query(
    "`업종다양성지수` <= 2 and `월평균 거래건수` >= 18 and `거래활동기간` >= 20 and `거래 연속성 비율` >= 70")
filtered_df

,고객ID,월평균 거래건수,최다가맹점업종명,업종다양성지수,거래 밀도 지수,거래활동기간,거래 연속성 비율,총거래건수
87449,3758688543,24,인터넷P/G,2,24.79,36,100,892
113757,3976463914,24,편 의 점,2,24.05,21,100,505
114368,3981970984,22,비영리/대상,2,22.33,30,100,670
126848,4085503325,20,기타 전문점,1,20.97,36,100,755
145708,4236134210,22,기타 전문점,1,22.08,36,100,795
148244,4257743638,21,인터넷P/G,2,21.16,28,100,592
150297,4274929235,24,인터넷P/G,2,24.52,21,100,515
168478,4414659119,18,인터넷P/G,2,18.96,36,100,682
184533,4537050673,20,인터넷P/G,1,20.28,36,100,730
227735,4867178040,19,비영리/대상,1,19.44,36,100,700


In [63]:
filtered_df.to_csv('부거래.csv', encoding='utf-8', index=False)